Import neccesary packages

In [1]:
import torch as pt
import datasets
from datasets import load_dataset

Here I will load the "squad" dataset first to see the data format before I process the spoken_squad data which is not available on datasets library.

In [2]:
raw_datasets = load_dataset("squad")

Found cached dataset squad (/home/mengkel/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [4]:
print("Context: ", raw_datasets["train"][0]["context"],'\n')
print("Question: ", raw_datasets["train"][0]["question"], '\n')
print("Answer: ", raw_datasets["train"][0]["answers"], '\n')
print("Id: ", raw_datasets["train"][0]["id"])

Context:  Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary. 

Question:  To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? 

Answer:  {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]} 

Id:  5733be284776f41900661182


In [5]:
from transformers import AutoTokenizer

In [6]:
#model_checkpoint = "bert-base-cased"
model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [7]:
tokenizer.is_fast

True

In [8]:
context = raw_datasets["train"][0]["context"]
question = raw_datasets["train"][0]["question"]

inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

'[CLS] to whom did the virgin mary allegedly appear in 1858 in lourdes france? [SEP] architecturally, the school has a catholic character. atop the main building\'s gold dome is a golden statue of the virgin mary. immediately in front of the main building and facing it, is a copper statue of christ with arms upraised with the legend " venite ad me omnes ". next to the main building is the basilica of the sacred heart. immediately behind the basilica is the grotto, a marian place of prayer and reflection. it is a replica of the grotto at lourdes, france where the virgin mary reputedly appeared to saint bernadette soubirous in 1858. at the end of the main drive ( and in a direct line that connects through 3 statues and the gold dome ), is a simple, modern stone statue of mary. [SEP]'

Download the spoken_train and spoken_test data from the link provided in the hw slide.

In [9]:
from datasets import load_dataset
dataset = load_dataset('json', data_files='spoken_train-v1.1.json', field = "data")

Found cached dataset json (/home/mengkel/.cache/huggingface/datasets/json/default-ece21904abacfb66/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
dataset["train"]

Dataset({
    features: ['title', 'paragraphs'],
    num_rows: 442
})

Here is the steps to process the raw data. 

In [11]:
def get_data(dat_pre):
    qid_lst, title_lst, context_lst, question_lst, answer_lst = [], [], [], [], []
    DEBUG_RUN = False
    for i, dat in enumerate(dat_pre):
        if DEBUG_RUN and len(context_lst) > 5000:
            break
        title = dat['title']
        for para in dat['paragraphs']:
            context = para['context']
            for qa in para['qas']:
                question = qa['question']
                q_id = qa['id']

                answer_text = qa['answers'][0]['text']
                answer_start = qa['answers'][0]['answer_start']
                assert type(answer_start)==int and answer_start >= 0
                #assert context[answer_start:answer_start + len(answer_text)] == answer_text

                part_before = context[:answer_start] + ' @@@@@ '
                part_after = ' @@@@@ ' + context[answer_start + len(answer_text):]
                new_context = part_before + answer_text + part_after

                qid_lst.append(q_id)
                title_lst.append(title)
                context_lst.append(new_context)
                question_lst.append(question)
                answer_lst.append([answer_text, answer_start])
    return qid_lst, title_lst, context_lst, question_lst, answer_lst 
            

I firstly did the data processing for the training dataset.

In [12]:
prod_data = {"train": []}

In [13]:
qid_lst, title_lst, context_lst, question_lst, answer_lst = get_data(dataset["train"])

In [14]:
print(len(qid_lst))

37111


In [15]:
for i in range(len(qid_lst)):
    prod_data["train"].append({'id': qid_lst[i], 'title': title_lst[i], 'context': context_lst[i], \
    'question': question_lst[i], 'answers': {'text': [answer_lst[i][0]], 'answer_start': [answer_lst[i][1]]}, "split": "train"})

In [16]:
print("Context: ", prod_data["train"][0]["context"],'\n')
print("Question: ", prod_data["train"][0]["question"], '\n')
print("Answer: ", prod_data["train"][0]["answers"], '\n')
print("Id: ", prod_data["train"][0]["id"])

Context:  architecturally the school has a catholic character. atop the main building school dome is the golden statue of the virgin mary. immediately in front of the main building in facing it is  @@@@@ a copper statue of christ @@@@@  with arms appraised with the legend and the bad meow names. next to the main building is the basilica of the sacred heart. immediately behind the basilica is the grotto im mary in place of prayer and reflection. it is a replica of the grotto at lourdes france where the virgin mary reputedly appeared to st bernadette still burning eighteen fifty eight. at the end of the main drive and in a direct line that connects through three statues in the gold dome is as simple modern stone statue of mary. 

Question:  What is in front of the Notre Dame Main Building? 

Answer:  {'text': ['a copper statue of christ'], 'answer_start': [187]} 

Id:  5733be284776f4190066117f


In [17]:
dataset1 = load_dataset('json', data_files='spoken_test-v1.1_WER54.json', field = "data")

Found cached dataset json (/home/mengkel/.cache/huggingface/datasets/json/default-983f0672be846a2e/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


  0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
dataset1

DatasetDict({
    train: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 48
    })
})

In [19]:
dataset1['train']

Dataset({
    features: ['title', 'paragraphs'],
    num_rows: 48
})

In [20]:
qid_lst1, title_lst1, context_lst1, question_lst1, answer_lst1 = get_data(dataset1['train'])

In [21]:
print(len(qid_lst1))

5351


In [22]:
prod_data["validation"]= []

In [23]:
for i in range(len(qid_lst1)):
    prod_data["validation"].append({'id': qid_lst1[i], 'title': title_lst1[i], 'context': context_lst1[i], \
    'question': question_lst1[i], 'answers': {'text': [answer_lst1[i][0]], 'answer_start': [answer_lst1[i][1]]}, "split": "validation"})

In [24]:
print("Context: ", prod_data["validation"][0]["context"],'\n')
print("Question: ", prod_data["validation"][0]["question"], '\n')
print("Answer: ", prod_data["validation"][0]["answers"], '\n')
print("Id: ", prod_data["validation"][0]["id"])

Context:  Two five oh fifty with an american football game to determine the champion of the national football league and f for the twenty fifteen feet then. The american football conferen @@@@@ Denver Broncos @@@@@ hampion denver broncos defeated the national football conference and effie champion carolina panthers twenty four to ten to earn their parents super bowl title. The game was played on february seventh one fifteen and revive the indian san francisco bay area santa clara california. If if with the fiftieth super bowl the league emphasize the golden anniversary with various gulf view the initiative as well as temporarily defending the tradition of naming each super bowl game with roman numeral find her with a game would have been known as super bowl felt that the logo of prominently featured the arabic numerals fifty. 

Question:  Which NFL team represented the AFC at Super Bowl 50? 

Answer:  {'text': ['Denver Broncos'], 'answer_start': [177]} 

Id:  56be4db0acb8001400a502ec


In [25]:
prod_data["train"][0]

{'id': '5733be284776f4190066117f',
 'title': 'University_of_Notre_Dame',
 'context': 'architecturally the school has a catholic character. atop the main building school dome is the golden statue of the virgin mary. immediately in front of the main building in facing it is  @@@@@ a copper statue of christ @@@@@  with arms appraised with the legend and the bad meow names. next to the main building is the basilica of the sacred heart. immediately behind the basilica is the grotto im mary in place of prayer and reflection. it is a replica of the grotto at lourdes france where the virgin mary reputedly appeared to st bernadette still burning eighteen fifty eight. at the end of the main drive and in a direct line that connects through three statues in the gold dome is as simple modern stone statue of mary.',
 'question': 'What is in front of the Notre Dame Main Building?',
 'answers': {'text': ['a copper statue of christ'], 'answer_start': [187]},
 'split': 'train'}

In [26]:
prod_data["validation"][0]

{'id': '56be4db0acb8001400a502ec',
 'title': 'Super_Bowl_50',
 'context': 'Two five oh fifty with an american football game to determine the champion of the national football league and f for the twenty fifteen feet then. The american football conferen @@@@@ Denver Broncos @@@@@ hampion denver broncos defeated the national football conference and effie champion carolina panthers twenty four to ten to earn their parents super bowl title. The game was played on february seventh one fifteen and revive the indian san francisco bay area santa clara california. If if with the fiftieth super bowl the league emphasize the golden anniversary with various gulf view the initiative as well as temporarily defending the tradition of naming each super bowl game with roman numeral find her with a game would have been known as super bowl felt that the logo of prominently featured the arabic numerals fifty.',
 'question': 'Which NFL team represented the AFC at Super Bowl 50?',
 'answers': {'text': ['Den

In [27]:
import json

In [28]:
with open("my_data.json", "w") as json_file:
    # write the dictionary to the file
    json.dump(prod_data, json_file)

In [29]:
train_data = load_dataset('json', data_files='my_data.json', field = 'train')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/mengkel/.cache/huggingface/datasets/json/default-4db658885caa2bb1/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [30]:
train_data

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers', 'split'],
        num_rows: 37111
    })
})

In [31]:
test_data = load_dataset('json', data_files='my_data.json', field = 'validation')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/mengkel/.cache/huggingface/datasets/json/default-c9383a109aecedbd/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [32]:
test_data

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers', 'split'],
        num_rows: 5351
    })
})

In [33]:
context = prod_data["train"][0]["context"]
question = prod_data["validation"][0]["question"]

In [34]:
inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

'[CLS] which nfl team represented the afc at super bowl 50? [SEP] architecturally the school has a catholic character. atop the main building school dome is the golden statue of the virgin mary. immediately in front of the main building in facing it is @ @ @ @ @ a copper statue of christ @ @ @ @ @ with arms appraised with the legend and the bad meow names. next to the main building is the basilica of the sacred heart. immediately behind the basilica is the grotto im mary in place of prayer and reflection. it is a replica of the grotto at lourdes france where the virgin mary reputedly appeared to st bernadette still burning eighteen fifty eight. at the end of the main drive and in a direct line that connects through three statues in the gold dome is as simple modern stone statue of mary. [SEP]'

In [35]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))

[CLS] which nfl team represented the afc at super bowl 50? [SEP] architecturally the school has a catholic character. atop the main building school dome is the golden statue of the virgin mary. immediately in front of the main building in facing it is @ @ @ @ @ a copper statue of christ @ @ @ @ @ with arms appraised with the legend and the bad meow names. next to the main building is the basilica of the sacred heart. immediately behind the basilica is the grotto [SEP]
[CLS] which nfl team represented the afc at super bowl 50? [SEP] @ @ @ @ a copper statue of christ @ @ @ @ @ with arms appraised with the legend and the bad meow names. next to the main building is the basilica of the sacred heart. immediately behind the basilica is the grotto im mary in place of prayer and reflection. it is a replica of the grotto at lourdes france where the virgin mary reputedly appeared to st bernadette still burning eighteen [SEP]
[CLS] which nfl team represented the afc at super bowl 50? [SEP] basili

In [36]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [37]:
answers = train_data["train"][2:6]["answers"]
start_positions = []
end_positions = []

for i, offset in enumerate(inputs["offset_mapping"]):
    sample_idx = inputs["overflow_to_sample_mapping"][i]
    answer = answers[sample_idx]
    start_char = answer["answer_start"][0]
    end_char = answer["answer_start"][0] + len(answer["text"][0])
    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label is (0, 0)
    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

start_positions, end_positions

([84, 48, 0], [85, 49, 0])

In [38]:
idx = 0
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["text"][0]

start = start_positions[idx]
end = end_positions[idx]
labeled_answer = tokenizer.decode(inputs["input_ids"][idx][start : end + 1])

print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")

Theoretical answer: twice, labels give: the basilica


In [39]:
idx = 1
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["text"][0]

decoded_example = tokenizer.decode(inputs["input_ids"][idx])
print(f"Theoretical answer: {answer}, decoded example: {decoded_example}")

Theoretical answer: twice, decoded example: [CLS] which nfl team represented the afc at super bowl 50? [SEP] @ @ @ @ a copper statue of christ @ @ @ @ @ with arms appraised with the legend and the bad meow names. next to the main building is the basilica of the sacred heart. immediately behind the basilica is the grotto im mary in place of prayer and reflection. it is a replica of the grotto at lourdes france where the virgin mary reputedly appeared to st bernadette still burning eighteen [SEP]


In [40]:
max_length = 384
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

some examples in this dataset have extra spaces at the beginning and the end that don’t add anything, so we removed those extra spaces.

To apply this function to the whole training set, we use the Dataset.map() method with the batched=True flag. It’s necessary here as we are changing the length of the dataset (since one example can give several training features):

In [41]:
train_dataset = train_data['train'].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=train_data['train'].column_names,
)

Map:   0%|          | 0/37111 [00:00<?, ? examples/s]

In [42]:
len(train_data["train"]), len(train_dataset)

(37111, 37361)

In [43]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [44]:
validation_dataset = test_data["train"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=test_data["train"].column_names,
)
len(test_data["train"]), len(validation_dataset)

Map:   0%|          | 0/5351 [00:00<?, ? examples/s]

(5351, 5444)

Fine-tuning the model with the Trainer API

In [45]:
small_eval_set = test_data["train"].select(range(100))
trained_checkpoint = "distilbert-base-cased-distilled-squad"

tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)
eval_set = small_eval_set.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=test_data["train"].column_names,
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [46]:
import torch
from transformers import AutoModelForQuestionAnswering

eval_set_for_model = eval_set.remove_columns(["example_id", "offset_mapping"])
eval_set_for_model.set_format("torch")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
batch = {k: eval_set_for_model[k].to(device) for k in eval_set_for_model.column_names}
trained_model = AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint).to(
    device
)

with torch.no_grad():
    outputs = trained_model(**batch)

In [47]:
start_logits = outputs.start_logits.cpu().numpy()
end_logits = outputs.end_logits.cpu().numpy()

In [48]:
import collections

example_to_features = collections.defaultdict(list)
for idx, feature in enumerate(eval_set):
    example_to_features[feature["example_id"]].append(idx)

In [49]:
import numpy as np

n_best = 20
max_answer_length = 30
predicted_answers = []

for example in small_eval_set:
    example_id = example["id"]
    context = example["context"]
    answers = []

    for feature_index in example_to_features[example_id]:
        start_logit = start_logits[feature_index]
        end_logit = end_logits[feature_index]
        offsets = eval_set["offset_mapping"][feature_index]

        start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
        end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
        for start_index in start_indexes:
            for end_index in end_indexes:
                # Skip answers that are not fully in the context
                if offsets[start_index] is None or offsets[end_index] is None:
                    continue
                # Skip answers with a length that is either < 0 or > max_answer_length.
                if (
                    end_index < start_index
                    or end_index - start_index + 1 > max_answer_length
                ):
                    continue

                answers.append(
                    {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                )

    best_answer = max(answers, key=lambda x: x["logit_score"])
    predicted_answers.append({"id": example_id, "prediction_text": best_answer["text"]})


In [50]:
theoretical_answers = [
    {"id": ex["id"], "answers": ex["answers"]} for ex in small_eval_set
]

In [51]:
print(predicted_answers[5])
print(theoretical_answers[5])

{'id': '56be8e613aeaaa14008c90d1', 'prediction_text': 'revive the indian san francisco bay area santa clara california'}
{'id': '56be8e613aeaaa14008c90d1', 'answers': {'answer_start': [487], 'text': ['"golden anniversary"']}}


In [52]:
#pip install evaluate

In [53]:
import evaluate

metric = evaluate.load("squad")

In [54]:
#print(metric)

In [55]:
metric.compute(predictions=predicted_answers, references=theoretical_answers)

{'exact_match': 39.0, 'f1': 50.56825396825397}

In [56]:
from tqdm.auto import tqdm


def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [57]:
compute_metrics(start_logits, end_logits, eval_set, small_eval_set)


  0%|          | 0/100 [00:00<?, ?it/s]

{'exact_match': 39.0, 'f1': 50.56825396825397}

Fine-tuning the model

In [58]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [59]:
from huggingface_hub import notebook_login

In [60]:
pip install ipywidgets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [61]:
notebook_login()

In [62]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-squad",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=False,
)

In [63]:
from transformers import Trainer

In [64]:
!pip install git-lfs

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The easy wayy to train it is to use the Trainer from transformers package

In [65]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
)
trainer.train()

/home/mengkel/.conda/envs/skln/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.837900
1000,0.321700
1500,0.261400
2000,0.237600
2500,0.207600
3000,0.185500
3500,0.176100
4000,0.178100
4500,0.150100
5000,0.118900


TrainOutput(global_step=14013, training_loss=0.14140971237099043, metrics={'train_runtime': 1290.7806, 'train_samples_per_second': 86.834, 'train_steps_per_second': 10.856, 'total_flos': 2.1965193288930816e+16, 'train_loss': 0.14140971237099043, 'epoch': 3.0})

In [66]:
# predictions, _, _ = trainer.predict(validation_dataset)
# start_logits, end_logits = predictions
# compute_metrics(start_logits, end_logits, validation_dataset, prod_data["validation"])

  0%|          | 0/5351 [00:00<?, ?it/s]

{'exact_match': 5.064473930106522, 'f1': 59.210471019732445}